In [5]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Create models directory
models_dir = "../models/normalmodels"
os.makedirs(models_dir, exist_ok=True)

# Load data
print("📊 Loading data...")
model_df = pd.read_csv("../data/processed/processed_weekly_modeling.csv")
ref_df = pd.read_csv("../data/processed/processed_weekly_reference.csv")

# Prepare data for Target_1w_Price
target_name = 'Target_1w_Price'
feature_cols = [col for col in model_df.columns if not col.startswith('Target_')]

X = model_df[feature_cols].copy()
y = model_df[target_name].copy()

# Train/test split
train_mask = ref_df['Is_Train'].values
X_train, X_test = X[train_mask].copy(), X[~train_mask].copy()
y_train, y_test = y[train_mask].copy(), y[~train_mask].copy()

# Clean features
for col in feature_cols:
    try:
        X_train[col] = pd.to_numeric(X_train[col])
        X_test[col] = pd.to_numeric(X_test[col])
    except:
        le = LabelEncoder()
        X_train[col] = le.fit_transform(X_train[col].astype(str))
        X_test[col] = le.transform(X_test[col].astype(str))

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Data prepared: {X_train.shape[1]} features, {X_train.shape[0]} train samples")

📊 Loading data...
✅ Data prepared: 64 features, 4080 train samples


In [6]:
print("\n" + "="*50)
print("TRAINING XGBOOST MODEL")
print("="*50)

import xgboost as xgb

# XGBoost model
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

xgb_model.fit(X_train_scaled, y_train)

# Save model
with open(os.path.join(models_dir, "xgboost_model.pkl"), 'wb') as f:
    pickle.dump(xgb_model, f)

print(f"✅ XGBoost model saved: ../models/normalmodels/xgboost_model.pkl")


TRAINING XGBOOST MODEL
✅ XGBoost model saved: ../models/normalmodels/xgboost_model.pkl


In [7]:
print("\n" + "="*50)
print("TRAINING LIGHTGBM MODEL")
print("="*50)

import lightgbm as lgb

# LightGBM model
lgb_model = lgb.LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

lgb_model.fit(X_train_scaled, y_train)

# Save model
with open(os.path.join(models_dir, "lightgbm_model.pkl"), 'wb') as f:
    pickle.dump(lgb_model, f)

print(f"✅ LightGBM model saved: ../models/normalmodels/lightgbm_model.pkl")


TRAINING LIGHTGBM MODEL
✅ LightGBM model saved: ../models/normalmodels/lightgbm_model.pkl


In [8]:
print("\n" + "="*50)
print("TRAINING CATBOOST MODEL")
print("="*50)

import catboost as cb

# CatBoost model
cat_model = cb.CatBoostRegressor(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=False
)

cat_model.fit(X_train_scaled, y_train)

# Save model
with open(os.path.join(models_dir, "catboost_model.pkl"), 'wb') as f:
    pickle.dump(cat_model, f)

print(f"✅ CatBoost model saved: ../models/normalmodels/catboost_model.pkl")


TRAINING CATBOOST MODEL
✅ CatBoost model saved: ../models/normalmodels/catboost_model.pkl


In [9]:
print("\n" + "="*50)
print("MODEL PERFORMANCE CHECK")
print("="*50)

from sklearn.metrics import mean_absolute_error, r2_score

models = {
    'XGBoost': xgb_model,
    'LightGBM': lgb_model,
    'CatBoost': cat_model
}

print(f"{'Model':<10} {'Test R²':<10} {'Test MAE':<10}")
print("-" * 30)

for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"{name:<10} {r2:.4f}     {mae:.2f}")

print("\n✅ All 3 models saved successfully!")
print("📁 Files created:")
print("   ../models/xgboost_model.pkl")
print("   ../models/lightgbm_model.pkl")
print("   ../models/catboost_model.pkl")


MODEL PERFORMANCE CHECK
Model      Test R²    Test MAE  
------------------------------
XGBoost    0.9049     4.50
LightGBM   0.9189     4.45
CatBoost   0.9437     4.18

✅ All 3 models saved successfully!
📁 Files created:
   ../models/xgboost_model.pkl
   ../models/lightgbm_model.pkl
   ../models/catboost_model.pkl


C:\Users\Arosha IIT\OneDrive - Robert Gordon University\Desktop\AIDS NOTES\DSGP\Wholesale Price Model\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
